In [ ]:
# Dependencies and Setup mike
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [ ]:
# Name of the CSV file
file = 

In [ ]:
# The correct encoding must be used to read the CSV in pandas
df = pd.read_csv(file)

In [ ]:
# Preview of the DataFrame
df.head()

In [ ]:
# Reduce dataframe to only relevant columns
reduced_df = df.loc[:, ["name", "goal", "pledged",
                                    "state", "country", "staff_pick", "backers_count", "spotlight"]]
reduced_df

In [ ]:
# Change column names to: Fire Name, Start Date, County, Acres Burned, Longitude, Latitude, Extinguished Date
print(df.rename(columns={'A': 'a', 'C': 'c'}))

In [ ]:
# Check data types
df.dtypes

In [ ]:
# Create a Jupyter Notebook illustrating the final data analysis

In [ ]:
# Where does it rain most frequently in California and is there a correlation with wildfires?

In [ ]:
# Is there a correlation between total amount of rain in a specific area and the total number of wildfires?

In [ ]:
# Is there a correlation between total number of acres burned and precipitation or the size of wildfires?

In [ ]:
# Is there a correlation between the duration of a wildfire and precipitation?

In [ ]:
# Do fires last longer when there is less precipitation? Could there be outside factors shortening the duration of a wildfire?